# Evaluate trained model

In [1]:
%load_ext autoreload
%autoreload 1

In [27]:
import os
import cv2

import sys
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image


try:
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
    import labinet.object_detect
    import labinet.box
except ImportError:
    # This part is only required to run the notebook
    # directory when the module itself is not installed.
    #
    # If you have the module installed, just use "import labinet..."
    import os
    import inspect
    # the .travis.yml is coded so that we execute tests from within test subdir. Relative to 'test' the .py is found in ../source/dev
    cmd_subfolder = os.path.realpath(os.path.abspath(os.path.join(os.path.split(inspect.getfile(inspect.currentframe()))[0], "..")))
    if cmd_subfolder not in sys.path:
        sys.path.insert(0, cmd_subfolder)
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
    import labinet.object_detect
    import labinet.box

%aimport labinet.io_util
%aimport labinet.object_detect
%aimport labinet.box

%matplotlib inline

## change path, model name etc to your needs

In [3]:
MODEL_NAME = 'inference_graph'   # the result from Step 6 Export Inference Graph
IMAGE_PATH = 'images'
IMAGE_NAME = '01-20190414173244-01.jpg'   # cmdline arg
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
CWD_PATH = os.path.join(os.getcwd(),'..')  # should become gitbase
NUM_CLASSES = 1

In [4]:
# model
PATH_TO_MODEL = os.path.join(CWD_PATH, MODEL_NAME, 'frozen_inference_graph.pb')
# label map
LABEL_MAP = os.path.join(CWD_PATH, 'data', 'object-detection.pbtxt')
# path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH, IMAGE_PATH, IMAGE_NAME)

## Load (frozen) Model into Memory

In [5]:
detection_graph = load_model(PATH_TO_MODEL)
#sess = tf.Session(graph=detection_graph)
   

### load labels 

In [6]:
categories, category_index = load_label_map(LABEL_MAP, NUM_CLASSES)

## Run Inference : Object Detection 
_load the image and run on it the object detection_

In [7]:
# load image
image =  Image.open(PATH_TO_IMAGE)
#imshow(np.asarray(image))

# inference
output_dict = labinet.object_detect.run_inference_for_single_image(image, detection_graph)

### Visualize result

In [13]:
image_with_boxes = labinet.object_detect.visualize_boxes_after_detection(image, output_dict, category_index)
# All the results have been drawn on image. Now display the image.
plt.figure(figsize=IMAGE_SIZE)
#plt.imshow(image_with_boxes)
print(IMAGE_NAME)


01-20190414173244-01.jpg


<Figure size 864x576 with 0 Axes>

In [14]:
boxes = output_dict['detection_boxes']
scores = output_dict['detection_scores']


In [15]:
%aimport labinet.object_detect 

boxes_to_use = labinet.object_detect.get_boxes_to_use(boxes, scores)
boxes_to_use[0]

array([0.6661899 , 0.25938419, 0.92370725, 0.6412617 ], dtype=float32)

In [16]:
%aimport labinet.box

In [17]:
boxes_normed = []
for box in boxes_to_use:
    box_normed = labinet.box.get_normalized_coordinates(box, image.size)
    boxes_normed.append(box_normed)

In [37]:
xml = labinet.box.box_to_labelimg_xml('file.jpg', (640,480), boxes_normed)
ET.dump(xml)
#xml.write("out.xml")

<annotation><folder>images</folder><filename>file.jpg</filename><path /><source><database>Unknown</database></source><size><width>480</width><height>640</height><depth>3</depth></size><segmented>0</segmented><object><name>dog</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>124</xmin><xmax>307</xmax><ymin>426</ymin><ymax>591</ymax></bndbox></object><object><name>dog</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>157</xmin><xmax>366</xmax><ymin>456</ymin><ymax>585</ymax></bndbox></object><object><name>dog</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>214</xmin><xmax>381</xmax><ymin>463</ymin><ymax>575</ymax></bndbox></object></annotation>
